In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install tensorflow_addons==0.11.2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pnd
from tqdm import tqdm
import sys
import math
import os
import zipfile
import six
import warnings
import random
import gc
import six
from math import ceil
from sklearn.model_selection import KFold, StratifiedKFold,train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from keras.metrics import top_k_categorical_accuracy


import cv2
import imgaug as ia
from imgaug import augmenters as iaa
# from moviepy.editor import VideoFileClip
from PIL import Image

from tensorflow.keras.utils import to_categorical

from scipy.io import loadmat
import keras.backend as K
from keras.engine.topology import get_source_inputs
from keras.layers import Activation
from keras.layers import AveragePooling3D
from keras.layers import BatchNormalization
from keras.layers import Conv3D
from keras.layers import Conv3DTranspose
from keras.layers import Dense
from keras.layers import Dropout,Flatten
from keras.layers import GlobalAveragePooling3D
from keras.layers import GlobalMaxPooling3D
from keras.layers import Input
from keras.layers import MaxPooling3D
from keras.layers import Reshape
from keras.layers import UpSampling3D
from keras.layers import Concatenate
from tensorflow.keras.models import Model,load_model
from keras.regularizers import l2
!pip install git+https://www.github.com/keras-team/keras-contrib.git
from keras_contrib.layers import SubPixelUpscaling
from tensorflow.keras.callbacks import LearningRateScheduler
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import SGD,Adam

In [ ]:
! nvidia-smi

In [ ]:
data_road = np.load('input/brain4carsnumpy/data_road.npy') 
data_face = np.load('input/brain4carsnumpy/data_face.npy') 

data_road_asl = np.load('input/brain4carsnumpy/data_road_asl.npy')
data_face_asl = np.load('input/brain4carsnumpy/data_face_asl.npy')

labels = np.load('input/brain4carsnumpy/labels_road.npy')

In [ ]:
from sklearn.utils import shuffle
data_road , data_road_asl ,data_face , data_face_asl,labels = shuffle(data_road , data_road_asl ,data_face , data_face_asl,labels,random_state=0 )

In [ ]:
def data_augmentation_tempen1(inputs,inputs1,inputs2,inputs3,y):
    trans_range=4
    temp_lst = []
    temp_lst1 = []
    temp_lst2 = []
    temp_lst3 = []
    yy = []
    # p0 = np.random.randint(-trans_range, trans_range + 1) + trans_range
    # p1 = np.random.randint(-trans_range, trans_range + 1) + trans_range


    if 1:
      if y==1:
        yy=3
      if y==3:
        yy=1
      if y==2:
        yy=4
      if y==4:
        yy=2
      for img in inputs:

          img = img[:, ::-1, :]
          p0 = np.random.randint(-trans_range, trans_range + 1) + trans_range
          p1 = np.random.randint(-trans_range, trans_range + 1) + trans_range
          img1 = img[p0:p0 + 128, p1:p1 + 128, :]
          img = cv2.resize(img1,(128,128))
          temp_lst.append(img)
      for img in inputs1:

          img = img[:, ::-1, :]
          p0 = np.random.randint(-trans_range, trans_range + 1) + trans_range
          p1 = np.random.randint(-trans_range, trans_range + 1) + trans_range 
          img1 = img[p0:p0 + 128, p1:p1 + 128, :]
          img = cv2.resize(img1,(128,128))
          temp_lst1.append(img)
      for img in inputs2:

          img = img[:, ::-1, :]
          p0 = np.random.randint(-trans_range, trans_range + 1) + trans_range
          p1 = np.random.randint(-trans_range, trans_range + 1) + trans_range
          img1 = img[p0:p0 + 128, p1:p1 + 128, :]
          img = cv2.resize(img1,(128,128))
          temp_lst2.append(img)
      for img in inputs3:

          img = img[:, ::-1, :]
          p0 = np.random.randint(-trans_range, trans_range + 1) + trans_range
          p1 = np.random.randint(-trans_range, trans_range + 1) + trans_range
          img1 = img[p0:p0 + 128, p1:p1 + 128, :]
          img = cv2.resize(img1,(128,128))
          temp_lst3.append(img)
      return np.array(temp_lst),np.array(temp_lst1),np.array(temp_lst2),np.array(temp_lst3),yy
    else:
      return inputs , inputs1 , inputs2 , inputs3 , y




In [ ]:
def data_augmentation_tempen(inputs,inputs1,inputs2,inputs3,y):
    trans_range=4
    temp_lst = []
    temp_lst1 = []
    temp_lst2 = []
    temp_lst3 = []
    yy = []
    # p0 = np.random.randint(-trans_range, trans_range + 1) + trans_range
    # p1 = np.random.randint(-trans_range, trans_range + 1) + trans_range


    if np.random.uniform() > 0.5:
      if y==1:
        yy=3
      if y==3:
        yy=1
      if y==2:
        yy=4
      if y==4:
        yy=2
      inputs  = np.pad(inputs, ((0, 0), (trans_range, trans_range), (trans_range, trans_range), (0, 0)), 'reflect')
      inputs1 = np.pad(inputs1, ((0, 0), (trans_range, trans_range), (trans_range, trans_range), (0, 0)), 'reflect')
      inputs2 = np.pad(inputs2, ((0, 0), (trans_range, trans_range), (trans_range, trans_range), (0, 0)), 'reflect')
      inputs3 = np.pad(inputs3, ((0, 0), (trans_range, trans_range), (trans_range, trans_range), (0, 0)), 'reflect')
      for img in inputs:

          img = img[:, ::-1, :]
          p0 = np.random.randint(-trans_range, trans_range + 1) + trans_range
          p1 = np.random.randint(-trans_range, trans_range + 1) + trans_range
          img1 = img[p0:p0 + 128, p1:p1 + 128, :]
          img = cv2.resize(img1,(128,128))
          temp_lst.append(img)
      for img in inputs1:

          img = img[:, ::-1, :]
          p0 = np.random.randint(-trans_range, trans_range + 1) + trans_range
          p1 = np.random.randint(-trans_range, trans_range + 1) + trans_range 
          img1 = img[p0:p0 + 128, p1:p1 + 128, :]
          img = cv2.resize(img1,(128,128))
          temp_lst1.append(img)
      for img in inputs2:

          img = img[:, ::-1, :]
          p0 = np.random.randint(-trans_range, trans_range + 1) + trans_range
          p1 = np.random.randint(-trans_range, trans_range + 1) + trans_range
          img1 = img[p0:p0 + 128, p1:p1 + 128, :]
          img = cv2.resize(img1,(128,128))
          temp_lst2.append(img)
      for img in inputs3:

          img = img[:, ::-1, :]
          p0 = np.random.randint(-trans_range, trans_range + 1) + trans_range
          p1 = np.random.randint(-trans_range, trans_range + 1) + trans_range
          img1 = img[p0:p0 + 128, p1:p1 + 128, :]
          img = cv2.resize(img1,(128,128))
          temp_lst3.append(img)
      return np.array(temp_lst),np.array(temp_lst1),np.array(temp_lst2),np.array(temp_lst3),yy
    else:
      return inputs , inputs1 , inputs2 , inputs3 , y




In [ ]:
def mixup_data(imgf,imgr,imgfa,imgra):

    lmbda= np.random.beta(1., 1.) 
    
    rv= np.random.permutation(30) 
    imgf = lmbda * imgf + (1- lmbda) * imgf[rv] 
    imgr = lmbda * imgr + (1- lmbda) * imgr[rv] 
    imgfa= lmbda * imgfa + (1- lmbda) * imgfa[rv] 
    imgra= lmbda * imgra + (1- lmbda) * imgra[rv] 

    # y_lmix= lmbda * y_l + (1- lmbda) * y_l[rv]
    
    return imgf,imgr,imgfa,imgra

import tensorflow_addons as tfa
def augment_cutout(image):
  image = tf.convert_to_tensor(image)
  p0 = np.random.randint(0, 15)*2
  p1 = np.random.randint(0, 15)*2
  img = tfa.image.random_cutout(image, (p0,p1), constant_values = 0)
  
  return img

In [ ]:
from keras.layers.merge import add
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K

!pip install keras_applications==1.0.7
!pip install classification-models-3D
!pip install efficientnet-3D
!pip install keras-self-attention
!git clone https://github.com/uzaymacar/attention-mechanisms.git
sys.path.append('./attention-mechanisms')
from layers import Attention

In [ ]:
# use it

import keras
from keras_applications.imagenet_utils import _obtain_input_shape
from keras_applications.imagenet_utils import preprocess_input as _preprocess_input
from classification_models_3D.tfkeras import Classifiers
from keras.layers.noise import GaussianNoise
from keras.models import Model
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate
from keras.regularizers import l2 # L2-regularisation
from keras.metrics import top_k_categorical_accuracy
from keras_self_attention import SeqSelfAttention, SeqWeightedAttention



In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K


def _bernoulli(shape, mean):
    return tf.nn.relu(tf.sign(mean - tf.random.uniform(shape, minval=0, maxval=1, dtype=tf.float32)))


class DropBlock2D(tf.keras.layers.Layer):
    def __init__(self, keep_prob, block_size, scale=True,name=None, **kwargs):
        super(DropBlock2D, self).__init__(name="DropBlock2D")
        self.keep_prob = float(keep_prob) if isinstance(keep_prob, int) else keep_prob
        self.block_size = int(block_size)
        self.names = name
        self.scale = tf.constant(scale, dtype=tf.bool) if isinstance(scale, bool) else scale
        super(DropBlock2D, self).__init__(**kwargs)
        
    def get_config(self):
        config = super().get_config().copy()
        config.update( {"block_size": self.block_size,"keep_prob": self.keep_prob,"name": self.names })

        
        
        return config

    def compute_output_shape(self, input_shape):
        return input_shape

    def build(self, input_shape):
        assert len(input_shape) == 4
        _, self.h, self.w, self.channel = input_shape.as_list()
        # pad the mask
        p1 = (self.block_size - 1) // 2
        p0 = (self.block_size - 1) - p1
        self.padding = [[0, 0], [p0, p1], [p0, p1], [0, 0]]
        self.set_keep_prob()
        super(DropBlock2D, self).build(input_shape)

    def call(self, inputs, training=None, **kwargs):
        def drop():
            mask = self._create_mask(tf.shape(inputs))
            output = inputs * mask
            output = tf.cond(self.scale,
                             true_fn=lambda: output *tf.cast(tf.size(mask), dtype=tf.float32)  / tf.reduce_sum(mask),
                             false_fn=lambda: output)
            return output

        if training is None:
            training = K.learning_phase()
        output = tf.cond(tf.logical_or(tf.logical_not(training), tf.equal(self.keep_prob, 1.0)),
                         true_fn=lambda: inputs,
                         false_fn=drop)
        return output

    def set_keep_prob(self, keep_prob=None):
        """This method only supports Eager Execution"""
        if keep_prob is not None:
            self.keep_prob = keep_prob
        w, h = tf.cast(self.w, dtype=tf.float32), tf.cast(self.h, dtype=tf.float32)
        
        self.gamma = (1. - self.keep_prob) * (w * h) / (self.block_size ** 2) / \
                     ((w - self.block_size + 1) * (h - self.block_size + 1))

    def _create_mask(self, input_shape):
        sampling_mask_shape = tf.stack([input_shape[0],
                                       self.h - self.block_size + 1,
                                       self.w - self.block_size + 1,
                                       self.channel])
        mask = _bernoulli(sampling_mask_shape, self.gamma)
        mask = tf.pad(mask, self.padding)
        mask = tf.nn.max_pool(mask, [1, self.block_size, self.block_size, 1], [1, 1, 1, 1], 'SAME')
        mask = 1 - mask
        return mask


class DropBlock3D(tf.keras.layers.Layer):
    def __init__(self, keep_prob, block_size, scale=True,name=None, **kwargs):
        super(DropBlock3D, self).__init__(name="DropBlock3D")
        self.keep_prob = float(keep_prob) if isinstance(keep_prob, int) else keep_prob
        self.block_size = int(block_size)
        self.scale = tf.constant(scale, dtype=tf.bool) if isinstance(scale, bool) else scale
        super(DropBlock3D, self).__init__(**kwargs)

    def compute_output_shape(self, input_shape):
        return input_shape

    def build(self, input_shape):
        assert len(input_shape) == 5
        _, self.d, self.h, self.w, self.channel = input_shape.as_list()
        # pad the mask
        p1 = (self.block_size - 1) // 2
        p0= (self.block_size - 1) - p1
        self.padding = [[0, 0], [p0, p1], [p0, p1], [p0, p1], [0, 0]]
        self.set_keep_prob()
        super(DropBlock3D, self).build(input_shape)

    def call(self, inputs, training=None, **kwargs):
        def drop():
            mask = self._create_mask(tf.shape(inputs))
            output = inputs * mask
            output = tf.cond(self.scale,
                             true_fn=lambda: output *tf.cast(tf.size(mask), dtype=tf.float32)  / tf.reduce_sum(mask),
                             false_fn=lambda: output)
            return output

        if training is None:
            training = K.learning_phase()
        output = tf.cond(tf.logical_or(tf.logical_not(training), tf.equal(self.keep_prob, 1.0)),
                         true_fn=lambda: inputs,
                         false_fn=drop)
        return output

    def set_keep_prob(self, keep_prob=None):
        """This method only supports Eager Execution"""
        if keep_prob is not None:
            self.keep_prob = keep_prob
        d, w, h = tf.cast(self.d, dtype=tf.float32), tf.cast(self.w, dtype=tf.float32), tf.cast(self.h, dtype=tf.float32)
        self.gamma = ((1. - self.keep_prob) * (d * w * h) / (self.block_size ** 3) /
                      ((d - self.block_size + 1) * (w - self.block_size + 1) * (h - self.block_size + 1)))

    def _create_mask(self, input_shape):
        sampling_mask_shape = tf.stack([input_shape[0],
                                        self.d - self.block_size + 1,
                                        self.h - self.block_size + 1,
                                        self.w - self.block_size + 1,
                                        self.channel])
        mask = _bernoulli(sampling_mask_shape, self.gamma)
        mask = tf.pad(mask, self.padding)
        mask = tf.nn.max_pool3d(mask, [1, self.block_size, self.block_size, self.block_size, 1], [1, 1, 1, 1, 1], 'SAME')
        mask = 1 - mask
        return mask

In [ ]:
# use it

import keras


from keras.models import Model
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate,AveragePooling2D,TimeDistributed
from keras.regularizers import l2 # L2-regularisation
from keras.metrics import top_k_categorical_accuracy
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.applications import Xception
import tensorflow as tf 
import random
import os
seed_value= 0
os.environ['PYTHONHASHSEED']=str(seed_value)
random.seed(seed_value)
np.random.seed(seed_value)
tf.random.set_seed(seed_value)


def define_model(input_shape):

    road = Input(shape=(128, 128,3))
    face = Input(shape=(128, 128,3))
#     road_opt = Input(shape=(128, 128,3))
#     face_opt = Input(shape=(128, 128,3))

    # road = Input(shape=(128, 384))
    road_opt = Input(shape=(128, 384))
    # face = Input(shape=(128, 384))
    face_opt = Input(shape=(128, 384))

    # road     = Input(shape=( 384,128))
    # road_opt = Input(shape=( 384,128))
    # face     = Input(shape=( 384,128))
    # face_opt = Input(shape=( 384,128))
    # road_opt = Input(shape=(15,128, 128,3))
    # face_opt = Input(shape=(15,128, 128,3))


    model1 = DenseNet121(include_top=False, weights='imagenet', input_tensor=road, input_shape=None, pooling=None, classes=1000)
    model2 = DenseNet121(include_top=False, weights='imagenet', input_tensor=face, input_shape=None, pooling=None, classes=1000)
    # model3 = DenseNet121(include_top=False, weights='imagenet', input_tensor=road_opt, input_shape=None, pooling=None, classes=1000)
    # model4 = DenseNet121(include_top=False, weights='imagenet', input_tensor=face_opt, input_shape=None, pooling=None, classes=1000)
    # base1 = Inception_Inflated3d(include_top=False,weights=None,input_tensor=None,input_shape=(15, 128,128,3),dropout_prob=0.0,endpoint_logit=False,classes=12)
    # base2 = Inception_Inflated3d(include_top=False,weights=None,input_tensor=None,input_shape=(15, 128,128,3),dropout_prob=0.0,endpoint_logit=False,classes=12)

    model1.trainable = True
    model2.trainable = True
    # model3.trainable = True
    # model4.trainable = True
    model1._name ='de'
    model2._name = 'dr'
    # base1._name ='dx'
    # base2._name = 'dc'
    # model3._name ='dee'
    # model4._name = 'dre'
    for layer in model2.layers:
      layer._name = layer.name + str("_2")
    # for layer in base2.layers:
    #   layer._name = layer.name + str("_2")
    # for layer in model3.layers:
    #   layer._name = layer.name + str("_21")
    # for layer in model4.layers:
    #   layer._name = layer.name + str("_22")


    regularizer = tf.keras.regularizers.l2(0.0001)

    # for layer in model2.layers:
    #     for attr in ['kernel_regularizer']:
    #         if hasattr(layer, attr):
    #           setattr(layer, attr, regularizer)
    # for layer in model1.layers:
    #     for attr in ['kernel_regularizer']:
    #         if hasattr(layer, attr):
    #           setattr(layer, attr, regularizer)
    x1 = model1.get_layer(model1.layers[-1].name).output
    x2 = model2.get_layer(model2.layers[-1].name).output
    # x2 = model2(face)
    print(x2.shape)
    # x3 = model3.get_layer(model3.layers[-1].name).output
    # x4 = model4.get_layer(model4.layers[-1].name).output
    x2 = DropBlock2D(keep_prob=0.5, block_size=5)(x2,training=trainphase)
    x1 = DropBlock2D(keep_prob=0.5, block_size=5)(x1,training=trainphase)

    x2 = AveragePooling2D((4,4))(x2)
    x1 = AveragePooling2D((4,4))(x1)
    # x2 = TimeDistributed(model1)(road)
    # x2 = TimeDistributed(model2)(face)
    # print(x2.shape)
    # x3 = AveragePooling2D((4,4))(x3)
    # x4 = AveragePooling2D((4,4))(x4)
    # print(AveragePooling2D((4,4))(x1).shape)
    feat = 128
    feat1 = 128
    feat2 = 512
    feat3 = 64
    numimg = 15
    ####### Feature Extraact From all frame input
    # out2 = tf.keras.layers.Embedding(256, 64, input_length=10)
    # out4 = tf.keras.layers.Embedding(256, 64, input_length=10)
    # out2 = base1(road_opt)
    # out4 = base2(face_opt)
    # road_opt = K.reshape(road_opt,(-1,128, 384))
    # face_opt = K.reshape(face_opt,(-1,128, 384))

    whole_seq_output2, final_memory_state2, final_carry_state2 = tf.keras.layers.LSTM(feat, return_sequences=True, return_state=True,dropout=0.0, recurrent_dropout=0.0)(road_opt) 
    whole_seq_output4, final_memory_state4, final_carry_state4 = tf.keras.layers.LSTM(feat, return_sequences=True, return_state=True,dropout=0.0, recurrent_dropout=0.0)(face_opt)
    # out2 = base1(road_opt)
    # out4 = base2(face_opt)
    # print(out2.shape)
    out1 = K.reshape(x1,(-1, numimg,1024))
    out2 = K.reshape(final_carry_state2,(-1, numimg,feat))
    

#     out2 = K.reshape(whole_seq_output2,(-1, feat,numimg*feat))
    # out22 = K.reshape(out2,(-1,1024))


    # out2 = K.reshape(x3,(-1, numimg,1024))

    out3 = K.reshape(x2,(-1, numimg,1024))
    out4 = K.reshape(final_carry_state4,(-1, numimg,feat))
    # out4 = K.reshape(whole_seq_output4,(-1, feat,numimg*feat))
    # out44 = K.reshape(out4,(-1,1024))


    # out4 = K.reshape(x4,(-1, numimg,1024))
    # out1 = Concatenate(axis=-2)([out1, out3])


    whole_seq_outpu1, final_memory_stat1, final_carry_stat1 = tf.keras.layers.LSTM(feat2, return_sequences=True, return_state=True,dropout=0.0)(out1)
    whole_seq_outpu2, final_memory_stat2, final_carry_stat2 = tf.keras.layers.LSTM(feat2, return_sequences=True, return_state=True,dropout=0.0)(out2) 
    whole_seq_outpu3, final_memory_stat3, final_carry_stat3 = tf.keras.layers.LSTM(feat2, return_sequences=True, return_state=True,dropout=0.0)(out3)
    whole_seq_outpu4, final_memory_stat4, final_carry_stat4 = tf.keras.layers.LSTM(feat2, return_sequences=True, return_state=True,dropout=0.0)(out4)

    # whole_seq_outpu1, final_memory_stat1, final_carry_stat1 = tf.keras.layers.LSTM(feat2, return_sequences=True, return_state=True,dropout=0.1)(whole_seq_outpu1)
    # whole_seq_outpu2, final_memory_stat2, final_carry_stat2 = tf.keras.layers.LSTM(feat1, return_sequences=True, return_state=True,dropout=0.0)(whole_seq_outpu2) 
    # whole_seq_outpu3, final_memory_stat3, final_carry_stat3 = tf.keras.layers.LSTM(feat2, return_sequences=True, return_state=True,dropout=0.1)(whole_seq_outpu3)
    # whole_seq_outpu4, final_memory_stat4, final_carry_stat4 = tf.keras.layers.LSTM(feat1, return_sequences=True, return_state=True,dropout=0.0)(whole_seq_outpu4)


    # merge_model1 = SeqWeightedAttention()(whole_seq_outpu1)
    # merge_model2 = SeqWeightedAttention()(whole_seq_outpu2)
    # merge_model3 = SeqWeightedAttention()(whole_seq_outpu3)
    # merge_model4 = SeqWeightedAttention()(whole_seq_outpu4)

    # out = Concatenate(axis=-1)([merge_model1, merge_model2,merge_model3,merge_model4])
    
    out = Concatenate(axis=-1)([final_carry_stat1, final_carry_stat2,final_carry_stat3,final_carry_stat4])
    # out1 = Concatenate(axis=-1)([final_carry_stat2,final_carry_stat4])
#     out = Concatenate(axis=-1)([final_carry_stat1,final_carry_stat3])


    # output =  Concatenate(axis=-1)([out1, out3])

    merge_model = Flatten(name='flatten1')(out)

    # merge_model1 = Flatten(name='flatten11')(output)

    # merge_model = Dense(512, activation='relu')(merge_model)
    # merge_model = Dropout(0.45)(merge_model)

    # merge_model1 = Dense(512, activation='relu')(merge_model1)
    # merge_model = Dropout(0.25)(merge_model)

    # merge_model =  Concatenate(axis=-1)([merge_model, merge_model1])

    # predictions = Dense(5,activation="softmax")(merge_model)


    # merge_model = Flatten(name='flatten1')(out1)

    # # merge_model1 = Flatten(name='flatten11')(output)

    # merge_model = Dense(512, activation='relu')(merge_model)
    # # merge_model = Dropout(0.45)(merge_model)

    # # merge_model1 = Dense(512, activation='relu')(merge_model1)
    # merge_model = Dropout(0.25)(merge_model)

    # # merge_model =  Concatenate(axis=-1)([merge_model, merge_model1])

    # predictions1 = Dense(5,activation="softmax")(merge_model)

    # if tf.keras.backend.max(predictions1,axis = 0) >tf.keras.backend.max(predictions,axis=0):#tf.keras.layers.Lambda(custom_layer, name="lambda_layer")([predictions1,predictions]):
    #   predictions = predictions1
    # def custom_layer1(tensor):
    #   return tensor
    # predictions = tf.cond(tf.keras.backend.max(predictions1,axis = 0) >tf.keras.backend.max(predictions,axis=0), lambda: custom_layer1(predictions1), lambda: custom_layer1(predictions))



    
    model = Model(inputs= [road,road_opt,face,face_opt], outputs=[merge_model],name ='kfj')

    return model

In [ ]:

def define_modelfc(input_shape):
    road = Input(shape=(2048,))
    merge_model = Dense(512, activation='relu')(road)
    merge_model = Dropout(0.45)(merge_model)

    predictions = Dense(5,activation="softmax")(road)    
    model = Model(inputs= [road], outputs=[predictions],name ='fc')

    return model

In [ ]:
# @tf.function(experimental_relax_shapes=True)
def train_step(images_a,images_b,sample_labels,epoch):

    # ratio = lambda_0 * (epoch / 300)
    ratio = .25 * (epoch / 160)

    with tf.GradientTape() as STtape:

        features = modela(images_a, training=True)
        y = modelb(features, training=True)
        isda.estimator.update_CV(np.array(features), np.argmax(sample_labels,axis=1))
        isda_aug_y = isda.isda_aug(modelb, features, y, np.argmax(sample_labels,axis=1), isda.estimator.CoVariance, ratio)
        supervised_lossb = CCLoss1(sample_labels,isda_aug_y)


    # isda.estimator.update_CV(np.array(features), np.argmax(sample_labels,axis=1))

    # with tf.GradientTape() as STtape1:

        
    #     y = modelb(features, training=True)
    #     isda_aug_y = isda.isda_aug(modelb, features, y, np.argmax(sample_labels,axis=1), isda.estimator.CoVariance, ratio)

       
    #     supervised_lossb = CCLoss1(sample_labels,isda_aug_y)
        

    variables = modela.trainable_variables + modelb.trainable_variables

    gradsa = STtape.gradient(supervised_lossb, variables)
    optimizer.apply_gradients(zip(gradsa,variables))

    # grads = STtape.gradient(supervised_lossb, modelb.trainable_variables)
    # optimizer.apply_gradients(zip(grads, modelb.trainable_variables))


    return  supervised_lossb
def evaluate():
    loss = 0

    st_accb = tf.keras.metrics.Accuracy()


    for i in range(num_train_data, num_test, batch_size):
       
        x_batch = []
        x_batch1 = []
        x_batch3 = []
        x_batch2 = []
        y_batch = []
        
        for idd in range(i,min(i+ batch_size,num_test)):
                # seq = get_seq()  
            imgf = data_face[idd]
            imgfa = data_face_asl[idd]
            imgr = data_road[idd]
            imgra = data_road_asl[idd]
            

            if labels[idd]==0 :#or y[idd]==1 or y[idd]==3 :
                
                  x_batch.append(imgf[0:30:2])
                  x_batch1.append(imgr[0:30:2] )
                  x_batch2.append(imgfa[0:30:2])
                  x_batch3.append(imgra [0:30:2])


                  y_batch.append(labels[idd])
            else:
                  a,b,c,d,e= data_augmentation_tempen(imgf[0:30:2],imgr[0:30:2],imgfa[0:30:2],imgra [0:30:2], labels[idd])
                  # a,b,c,d,e= data_augmentation_tempen(imgf,imgr,imgfa,imgra, y[idd])

                  x_batch.append(a)
                  x_batch1.append(b)
                  x_batch2.append(c)
                  x_batch3.append(d)
                  y_batch.append(e)



        x_batch  = np.array(x_batch).reshape((-1,128,384)) /255
        x_batch1 = np.array(x_batch1).reshape((-1,128,384))/255

        x_batch2 = np.array(x_batch2).reshape((-1,128,128,3))/255
        x_batch3 = np.array(x_batch3).reshape((-1,128,128,3))/255
        image_1 = [x_batch3,x_batch1,x_batch2,x_batch]
        y_batch = np.array(y_batch)
        y_batch = to_categorical(y_batch,5)

        features  = modela(image_1, training=False)
        logitsb = modelb(features , training=False)

        accuracyb = st_accb(tf.math.argmax(y_batch,axis=1), tf.math.argmax(logitsb,axis=1))
        loss += CCLoss(y_batch, logitsb).numpy()

    loss /= int(65/batch_size)
    accuracyb = accuracyb.numpy()
    return loss,accuracyb

In [ ]:
class EstimatorCV():
    def __init__(self, feature_num, class_num):
        super(EstimatorCV, self).__init__()
        self.class_num = class_num
        self.CoVariance = np.zeros((class_num, feature_num, feature_num))
        self.Ave = np.zeros((feature_num, class_num))
        self.Amount = np.zeros((class_num))

    def update_CV(self, features, labels):
        N = features.shape[0]
        C = self.class_num
        A = features.shape[1]

        # NxCxFeatures = features.reshape(N, 1, A).expand(N, C, A)
        # NxCxFeatures = np.broadcast_to(features.reshape((N, 1, A)),(N, C, A))
        NxCxFeatures = np.broadcast_to(features.reshape((N, A, 1)),(N, A, C))


        onehot = np.zeros((N, C))
        # onehot.scatter_(1, labels.reshape((-1, 1)), 1)#####
        onehot = scatter_numpy(onehot, 1, labels.reshape((-1, 1)), 1)

        # NxCxA_onehot = np.broadcast_to(onehot.reshape((N, C, 1)),(N, C, A))####
        NxCxA_onehot = np.broadcast_to(onehot.reshape((N, 1, C)),(N, A, C))####



        features_by_sort = np.multiply(NxCxFeatures,NxCxA_onehot)
        

        Amount_CxA = np.sum(NxCxA_onehot,axis=0)#NxCxA_onehot.sum(0)
        Amount_CxA[Amount_CxA == 0] = 1

        ave_CxA =  np.sum(features_by_sort,axis=0) / Amount_CxA#features_by_sort.sum(0) / Amount_CxA

        var_temp = features_by_sort -  np.multiply(np.broadcast_to(ave_CxA,(N, A, C)),NxCxA_onehot) ##

        



        var_temp = np.divide(np.transpose(var_temp,(2, 1, 0))@ np.transpose(var_temp,(2, 0, 1)),np.broadcast_to(Amount_CxA.reshape((C, A, 1)),(C, A, A)))

        # sum_weight_CV = onehot.sum(0).view(C, 1, 1).expand(C, A, A)
        sum_weight_CV = np.broadcast_to(np.sum(onehot,axis=0).reshape((C, 1, 1)),(C, A, A))


        # sum_weight_AV = onehot.sum(0).view(C, 1).expand(C, A)
        sum_weight_AV = np.broadcast_to(np.sum(onehot,axis=0).reshape((C, 1)),(C, A))


        weight_CV = np.broadcast_to(np.divide(sum_weight_CV,sum_weight_CV + self.Amount.reshape((C, 1, 1))),(C, A, A)).copy()#.setflags(write=1)#.copy().fill(5.446361E-01)#
       
        
        
        weight_CV[weight_CV != weight_CV] = 0
        

        weight_AV = np.broadcast_to(np.divide(sum_weight_AV,sum_weight_AV + self.Amount.reshape((C, 1))),(C, A)).copy()#.setflags(write=1)####
        weight_AV[np.where(weight_AV != weight_AV)[0]] = 0

      

        additional_CV = np.multiply(np.multiply(weight_CV,(1 - weight_CV)),((self.Ave - ave_CxA).reshape((C, A, 1))@(self.Ave - ave_CxA).reshape((C, 1, A))))####

        self.CoVariance = (np.multiply(self.CoVariance,(1 - weight_CV)) + 
                      np.multiply(var_temp,weight_CV)) + additional_CV
       




        self.Ave = (np.multiply(self.Ave,(1 - weight_AV.reshape((A,  C)))) + np.multiply(ave_CxA,weight_AV.reshape((A,  C))))
        self.Amount += np.sum(onehot,axis=0)


class ISDALoss():
    def __init__(self, feature_num, class_num):
        super(ISDALoss, self).__init__()

        self.estimator = EstimatorCV(feature_num, class_num)

        self.class_num = class_num

        self.cross_entropy = tf.keras.losses.CategoricalCrossentropy()

    def isda_aug(self, fc, features, y, labels, cv_matrix, ratio):

        N = features.shape[0]
        C = self.class_num
        A = features.shape[1]
        # C = features.shape[1]#self.class_num
        # A = self.class_num# features.shape[1]

        weight_m = list(fc.get_weights())[0]

        # print(weight_m.shape)

        # NxW_ij = np.broadcast_to(weight_m,(N, C, A)).copy()
        NxW_ij = np.broadcast_to(weight_m,(N, A, C)).copy()


        # NxW_kj = gather_numpy(NxW_ij, 1, np.broadcast_to(labels.reshape(N, 1, 1),(N, C, A)).copy())
        NxW_kj = gather_numpy(NxW_ij, 2, np.broadcast_to(labels.reshape(N, 1, 1),(N, A, C)).copy())


        CV_temp = cv_matrix[labels]

        # sigma2 = ratio * \
        #          torch.bmm(torch.bmm(NxW_ij - NxW_kj,
        #                              CV_temp).view(N * C, 1, A),
        #                    (NxW_ij - NxW_kj).view(N * C, A, 1)).view(N, C)
        # print(NxW_ij.shape) # (2, 512, 5)
        # print(NxW_kj.shape) # (2, 512, 5)
        # # print(((NxW_ij - NxW_kj)@CV_temp).shape)
        # print(CV_temp.shape) #    (2, 5, 512)@  (2, 512, 512)
        # # print(CV_temp.shape)3
        # print((np.transpose((NxW_ij - NxW_kj),(0, 2, 1))@CV_temp).shape) # (2, 5, 512)
        
        # print(np.transpose((NxW_ij - NxW_kj),(0, 2, 1)).shape)# (2, 5, 512)  (2, 512, 5)

        sigma2 = ratio * ((np.transpose((NxW_ij - NxW_kj),(0, 2, 1))@CV_temp)@ (NxW_ij - NxW_kj))


        # sigma2 = ratio * (((NxW_ij - NxW_kj)@CV_temp)@ np.transpose((NxW_ij - NxW_kj),(0, 2, 1)))

        sigma2 = np.sum(np.multiply(sigma2,np.broadcast_to(np.eye((C)),(N, C, C)).copy()),axis=2).reshape((N, C))

        aug_result = y + 0.5 * sigma2

        return aug_result



In [ ]:
def scatter_numpy(data, dim, index, src):
    """
    Writes all values from the Tensor src into self at the indices specified in the index Tensor.

    :param dim: The axis along which to index
    :param index: The indices of elements to scatter
    :param src: The source element(s) to scatter
    :return: self
    """
    if index.dtype != np.dtype('int_'):
        raise TypeError("The values of index must be integers")
    if data.ndim != index.ndim:
        raise ValueError("Index should have the same number of dimensions as output")
    if dim >= data.ndim or dim < -data.ndim:
        raise IndexError("dim is out of range")
    if dim < 0:
        # Not sure why scatter should accept dim < 0, but that is the behavior in PyTorch's scatter
        dim = data.ndim + dim
    idx_xsection_shape = index.shape[:dim] + index.shape[dim + 1:]
    self_xsection_shape = data.shape[:dim] + data.shape[dim + 1:]
    if idx_xsection_shape != self_xsection_shape:
        raise ValueError("Except for dimension " + str(dim) +
                         ", all dimensions of index and output should be the same size")
    if (index >= data.shape[dim]).any() or (index < 0).any():
        raise IndexError("The values of index must be between 0 and (self.shape[dim] -1)")

    def make_slice(arr, dim, i):
        slc = [slice(None)] * arr.ndim
        slc[dim] = i
        return slc

    # We use index and dim parameters to create idx
    # idx is in a form that can be used as a NumPy advanced index for scattering of src param. in self
    idx = [[*np.indices(idx_xsection_shape).reshape(index.ndim - 1, -1),
            index[make_slice(index, dim, i)].reshape(1, -1)[0]] for i in range(index.shape[dim])]
    idx = list(np.concatenate(idx, axis=1))
    idx.insert(dim, idx.pop())

    if not np.isscalar(src):
        if index.shape[dim] > src.shape[dim]:
            raise IndexError("Dimension " + str(dim) + "of index can not be bigger than that of src ")
        src_xsection_shape = src.shape[:dim] + src.shape[dim + 1:]
        if idx_xsection_shape != src_xsection_shape:
            raise ValueError("Except for dimension " +
                             str(dim) + ", all dimensions of index and src should be the same size")
        # src_idx is a NumPy advanced index for indexing of elements in the src
        src_idx = list(idx)
        src_idx.pop(dim)
        src_idx.insert(dim, np.repeat(np.arange(index.shape[dim]), np.prod(idx_xsection_shape)))
        data[idx] = src[src_idx]

    else:
        data[idx] = src

    return data


def gather_numpy(data, dim, index):
    """
    Gathers values along an axis specified by dim.
    For a 3-D tensor the output is specified by:
        out[i][j][k] = input[index[i][j][k]][j][k]  # if dim == 0
        out[i][j][k] = input[i][index[i][j][k]][k]  # if dim == 1
        out[i][j][k] = input[i][j][index[i][j][k]]  # if dim == 2

    :param dim: The axis along which to index
    :param index: A tensor of indices of elements to gather
    :return: tensor of gathered values
    """
    idx_xsection_shape = index.shape[:dim] + index.shape[dim + 1:]
    self_xsection_shape = data.shape[:dim] + data.shape[dim + 1:]
    if idx_xsection_shape != self_xsection_shape:
        raise ValueError("Except for dimension " + str(dim) +
                         ", all dimensions of index and self should be the same size")
    if index.dtype != np.dtype('int_'):
        raise TypeError("The values of index must be integers")
    data_swaped = np.swapaxes(data, 0, dim)
    index_swaped = np.swapaxes(index, 0, dim)
    # print(index_swaped.shape)
    # print(data_swaped.shape)

    gathered = np.choose(index_swaped, data_swaped)
    return np.swapaxes(gathered, 0, dim)

In [ ]:
from PIL import Image
from PIL import ImageOps
IMAGE_SIZE = 128

def int_parameter(level, maxval):
    return int(level * maxval / 10)


def float_parameter(level, maxval):
    return float(level) * maxval / 10.


def sample_level(n):
    return np.random.uniform(low=0.1, high=n)


def autocontrast(pil_img, _):
    return ImageOps.autocontrast(pil_img)


def equalize(pil_img, _):
    return ImageOps.equalize(pil_img)


def posterize(pil_img, level):
    level = int_parameter(sample_level(level), 4)
    return ImageOps.posterize(pil_img, 4 - level)


def rotate(pil_img, level):
    degrees = int_parameter(sample_level(level), 30)
    if np.random.uniform() > 0.5:
        degrees = -degrees
    return pil_img.rotate(degrees, resample=Image.BILINEAR)


def solarize(pil_img, level):
    level = int_parameter(sample_level(level), 256)
    return ImageOps.solarize(pil_img, 256 - level)


def shear_x(pil_img, level):
    level = float_parameter(sample_level(level), 0.3)
    if np.random.uniform() > 0.5:
        level = -level
    return pil_img.transform((IMAGE_SIZE, IMAGE_SIZE),
                            Image.AFFINE, (1, level, 0, 0, 1, 0),
                            resample=Image.BILINEAR)


def shear_y(pil_img, level):
    level = float_parameter(sample_level(level), 0.3)
    if np.random.uniform() > 0.5:
        level = -level
    return pil_img.transform((IMAGE_SIZE, IMAGE_SIZE),
                            Image.AFFINE, (1, 0, 0, level, 1, 0),
                            resample=Image.BILINEAR)


def translate_x(pil_img, level):
    level = int_parameter(sample_level(level), IMAGE_SIZE / 3)
    if np.random.random() > 0.5:
        level = -level
    return pil_img.transform((IMAGE_SIZE, IMAGE_SIZE),
                            Image.AFFINE, (1, 0, level, 0, 1, 0),
                            resample=Image.BILINEAR)


def translate_y(pil_img, level):
    level = int_parameter(sample_level(level), IMAGE_SIZE / 3)
    if np.random.random() > 0.5:
        level = -level
    return pil_img.transform((IMAGE_SIZE, IMAGE_SIZE),
                            Image.AFFINE, (1, 0, 0, 0, 1, level),
                            resample=Image.BILINEAR)
    
augmentations = [autocontrast, 
                  equalize, 
                  posterize, 
                  solarize]


def apply_op(image, op, severity):
    image = np.clip(image * 255., 0, 255).astype(np.uint8)
    pil_img = Image.fromarray(image)  
    pil_img = op(pil_img, severity)
    return np.asarray(pil_img).astype(np.float32) 


def augment_and_mix( image, severity=3, width=3, depth=-1, alpha=1.):

    ws = np.random.dirichlet([alpha] * width).astype(np.float32)
    m  = np.float32(np.random.beta(alpha, alpha))
    mix = np.zeros_like(image).astype(np.float32)

    for i in range(width):
        image_aug = image.copy()
        depth = depth if depth > 0 else np.random.randint(1, 3)
        for _ in range(depth):
            op = np.random.choice(augmentations)
            image_aug = apply_op(image_aug, op, severity)
           
            mix += ws[i] * image_aug

    
    mixed = (1 - m)*image + m*mix
    return mixed


def apply_augment(data,udata,dual=True):
    
    batch_len = len(data)  
    X_orig = np.zeros((batch_len, 128,128,3), dtype=np.float32)
    
    
    if dual:
        X_aug1 = np.zeros_like(X_orig, dtype=np.float32)
        X_aug2 = np.zeros_like(X_orig, dtype=np.float32)
        # X_aug3 = np.zeros_like(X_orig, dtype=np.float32)
    for  index in range(batch_len):
        img = data[index]
        uimg = udata[index]
        
        if dual:
            X_aug1[index] = augment_and_mix(uimg)
            X_aug2[index] = augment_and_mix(img)
            # X_orig[index] = augment_and_mix(img)
        else:
            X_orig[index] = augment_and_mix(img)
       
    
    if not dual:
        return X_orig
    else:
        return  X_aug1, X_aug2

In [ ]:
import time
num_train_data = 364
batch_size = 5
num_epoch = 160
num_test = 455
trainphase = True
modela = define_model((30,128,256,3))
modelb = define_modelfc((30,128,256,3))

# modela = tf.keras.models.load_model('/content/drive/MyDrive/Modelat.h5') 
# modelb = tf.keras.models.load_model('/content/drive/MyDrive/Modelbt.h5')

def step_decay(epoch):
     
     initAlpha = 0.0003
     factor = .89
     dropEvery = 39
     alpha = initAlpha * (factor ** np.floor((1 + epoch) / dropEvery))
     return float(alpha)

input_shape = [(None,128, 128, 3),(None,128, 384),(None,128, 128, 3),(None,128, 384)]
input_shape = [(None,2048)]


modela.build(input_shape)    
modelb.build(input_shape) 

CCLoss = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.25)   
CCLoss1 = tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.25)  
loss_object = tf.keras.losses.MeanSquaredError()
isda = ISDALoss(int(2048), 5)


optimizer = Adam(lr=0.0003, beta_1=0.9, beta_2=0.999, epsilon=None, amsgrad=False)
iStep = 0 
print('Training beginning')
total_epoch_loss = []

total_test_acc =[]
tmp =0
tmpb =0
tmpa =0
for epoch in range(num_epoch):
    epoch_loss = 0 # total epoch loss

    t1 = int(time.time())
    
    for i in range(0, num_train_data, batch_size):
        x_batch = []
        x_batch1 = []
        x_batch3 = []
        x_batch2 = []
        y_batch = []
        
        for idd in range(i,min(i + batch_size,num_train_data)):
              
            imgf = data_face[idd]
            imgfa = data_face_asl[idd]
            imgr = data_road[idd]
            imgra = data_road_asl[idd]
            
            if np.random.uniform() > 0.5:
               imgfa,imgra = apply_augment(imgfa,imgra)
            
            if np.random.uniform() > 0.5:
              imgfa = augment_cutout(imgfa).numpy()
              # imgra = augment_cutout(imgra).numpy()
            if np.random.uniform() > 0.5:
              # imgfa = augment_cutout(imgfa).numpy()
              imgra = augment_cutout(imgra).numpy()
          

            if labels[idd]==0 :#or y[idd]==1 or y[idd]==3 :
                
                  x_batch.append(imgf[0:30:2])
                  x_batch1.append(imgr[0:30:2] )
                  x_batch2.append(imgfa[0:30:2])
                  x_batch3.append(imgra [0:30:2])
                  # x_batch.append(imgf)
                  # x_batch1.append(imgr )
                  # x_batch2.append(imgfa)
                  # x_batch3.append( imgra )

                  y_batch.append(labels[idd])
            else:
                  if np.random.uniform() > 0.5:
                    a,b,c,d,e= data_augmentation_tempen(imgf[0:30:2],imgr[0:30:2],imgfa[0:30:2],imgra [0:30:2], labels[idd])
                    # a,b,c,d,e= data_augmentation_tempen(imgf,imgr,imgfa,imgra, y[idd])

                    x_batch.append(a)
                    x_batch1.append(b)
                    x_batch2.append(c)
                    x_batch3.append(d)
                    y_batch.append(e)
                  else:
                    a,b,c,d,e= data_augmentation_tempen(imgf[1:30:2],imgr[1:30:2],imgfa[1:30:2],imgra [1:30:2], labels[idd])
                    
                    x_batch.append(a)
                    x_batch1.append(b)
                    x_batch2.append(c)
                    x_batch3.append(d)
                    y_batch.append(e)



        x_batch  = np.array(x_batch).reshape((-1,128,384)) /255
        x_batch1 = np.array(x_batch1).reshape((-1,128,384))/255

        x_batch2 = np.array(x_batch2).reshape((-1,128,128,3))/255
        x_batch3 = np.array(x_batch3).reshape((-1,128,128,3))/255

        y_batch = np.array(y_batch)
        y_batch = to_categorical(y_batch,5)


       



        lr = step_decay(epoch)
        optimizer.learning_rate.assign(lr)

        image_1 = [x_batch3,x_batch1,x_batch2,x_batch]
        trainphase = True

        total_loss = train_step(image_1,image_1, y_batch,epoch)

        epoch_loss       += total_loss.numpy()

    epoch_loss       /= int(num_train_data/batch_size) # calc mean of epoch loss
   

    total_epoch_loss.append(epoch_loss)
    trainphase = False
    Te_test_loss,accuracyb = evaluate()

    if tmpb<accuracyb:
      tmpb = accuracyb
      t2 = int(time.time())
      modela.save('./Modela.h5')
      modelb.save('./Modelfc.h5')
      message = f"Epoch {epoch+1}/{num_epoch} Done: BAcc: {tmpb:.3f} | t: {t2-t1} sec"
      print(message)
    else:
      message = f"Epoch {epoch+1}/{num_epoch} Done: max Acc: {tmpb:.3f} |"
      print(message)
